In [1]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pathlib
import ipywidgets
import traitlets
import ipyelk

In [3]:
from rdflib import Graph
g = Graph()
g.parse("../graphs/python/sum3.ttl")
#g.parse("simple_sum.ttl")

print(len(g))

234


In [4]:
#METHODS
#=======

In [5]:
def find_layer_count():
    count_layer_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT * 
        WHERE {
            ?composition fnoc:start ?start .
        } 
        """ 

    count = 1
    for row in g.query(count_layer_query):
        #print(row.composition)
        count = count+1
    
    return count

In [6]:
def find_compositions():
    compositions_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT * 
        WHERE {
            ?composition a fno:Composition
        } 
        """ 

    compositions = []
    for row in g.query(compositions_query):
        compositions.append(row.composition)
    
    return compositions 

In [7]:
def find_start_node_per_layer(composition):
    start_node_for_composition = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?composition fnoc:represents ?rep .
            ?composition fnoc:start ?start .
        } 
        """    

    start_node_hierarchy = {}
    for row in g.query(start_node_for_composition, initBindings={'composition': composition}):
        start_node_hierarchy[row.rep] = row.start

    return start_node_hierarchy

In [8]:
def expand_layer_with_next_nodes(node):
    expand_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?node fnoc:next ?next .
        } 
        """

    for row in g.query(expand_query, initBindings={'node': node}):
        return row.next

In [9]:
def find_applies_for_node(node):
    applies_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?node fnoc:applies ?application .
        } 
        """

    for row in g.query(applies_query, initBindings={'node': node}):
        return node, row.application    

In [10]:
def expand_reused_composition(application, function, temp):
    #print(function)
    if function in node_hierarchy.keys():
        #print(node_hierarchy[function])
        temp[application] = node_hierarchy[function]

    return temp

In [11]:
def create_hier_tree(node_hierarchy, nodes_data):
    #print(node_hierarchy)
    #Create the necessary nodes
    s = set()
    #print(s)
    for k in node_hierarchy.keys():
        s.add(k)    
        for value in node_hierarchy[key]:
            s.add(value)
    #print(s)

    nodes = []
    for node in s:
        #print(node)
        nodes.append({"id" : node})

    #Do not ask me why, but without it, the graph is not plotted ... :-(
    nodes.append({"hidden":True,"id":"dummy"})
    
    #print(nodes)
    nodes_data["nodes"] = nodes

    links = []
    #Create the necessary links between the nodes (NOT ports)
    for k in node_hierarchy.keys():
        for val in node_hierarchy[k]:
            link = {"source": k, "target": val}
            links.append(link)
    nodes_data["links"] = links    

    return nodes_data

In [12]:
def create_node_ports_with_labels():
    labels = {}
    
    find_port_input_labels = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?mapping a fno:Mapping .
            ?mapping fno:parameterMapping ?paramMapping .
            ?paramMapping fnom:functionParameter ?functionParameter .
            ?paramMapping fnom:implementationProperty ?argument
        } 
        """ 

    for row in g.query(find_port_input_labels):
        print(row.paramMapping)
        print("\t"+row.functionParameter)
        print("\t"+row.argument)
        labels[row.functionParameter] = row.argument.toPython()

    find_port_output_labels = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?mapping a fno:Mapping .
            ?mapping fno:returnMapping ?returnMapping .
            ?returnMapping fnom:functionOutput ?argument
        } 
        """ 

    for row in g.query(find_port_output_labels):
        print(row.returnMapping)
        print("\t"+row.argument)
        labels[row.argument] = "out"    

    return labels

In [13]:
def find_mappings_and_create_links():
    links = []
    links = find_mappings_and_create_links_input(links)
    links = find_mappings_and_create_links_output_input(links)
    return links

In [14]:
def find_mappings_and_create_links_input(links):
    mapping_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?composition a fno:Composition .
            ?composition fnoc:composedOf ?mapping .
            ?mapping fnoc:mapFrom ?from .
            ?from fnoc:constituentFunction ?source .
            ?from fnoc:functionParameter ?sourcePort .
            ?mapping fnoc:mapTo ?to .
            ?to fnoc:constituentFunction ?target .
            ?to fnoc:functionParameter ?targetPort .            
        } 
        """
    for row in g.query(mapping_query):
        print("\nCOMPOSITION: ")
        print(row.composition)
        print("----------------------------------")
        print("MAPPING: "+row.mapping)
        print("sourcePort")
        print("\t"+row.sourcePort)
        print("targetPort")
        print("\t"+row.targetPort)
        print("source")
        print("\t"+row.source)
        print("target")
        print("\t"+row.target)
        links = create_link(links, row)

    print(links)
    return links

In [15]:
def find_mappings_and_create_links_output_input(links):
    mapping_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?composition a fno:Composition .
            ?composition fnoc:composedOf ?mapping .
            ?mapping fnoc:mapFrom ?from .
            ?from fnoc:constituentFunction ?source .
            ?from fnoc:functionOutput ?sourcePort .
            ?mapping fnoc:mapTo ?to .
            ?to fnoc:constituentFunction ?target .
            ?to fnoc:functionParameter ?targetPort .            
        } 
        """
    for row in g.query(mapping_query):
        print("\nCOMPOSITION: ")
        print(row.composition)
        print("----------------------------------")
        print("MAPPING: "+row.mapping)
        print("sourcePort")
        print("\t"+row.sourcePort)
        print("targetPort")
        print("\t"+row.targetPort)
        print("source")
        print("\t"+row.source)
        print("target")
        print("\t"+row.target)
        links = create_link(links, row)

    print(links)
    return links

In [16]:
def create_link(links, row):
    link = {}
    link["key"] = len(links)
    link["sourcePort"] = row.sourcePort
    link["targetPort"] = row.targetPort
    link["source"] = row.source
    link["target"] = row.target
    links.append(link)
    return links

In [17]:
from rdflib import URIRef, BNode, Literal

def check_and_percolate_dupplications(duplicate_function_mapping, links_data, node_hierarchy):
    #print(duplicate_function_mapping)
    #print()
    duplicated_links_to_be_added = []
    output_completed = []
    count = 1
    
    for key, value in duplicate_function_mapping.items():
        #print("CONSIDERING FOLLOWING KEY/VALUE PAIR")
        #print(key)
        #print(value)

        for link in links_data["links"]:
            #print("CHECKING LINK: ")
            #print(link)
            if link['target'] == value:
                #print("FOUND LINK:")
                #print(link)

                ##Find the correct source reference in node_hierarchy
                for kk, vv in node_hierarchy.items():
                    #print("CHECKING NODE HIERARCHY")
                    #print(kk)
                    for m in vv:
                        #print(m)
                        if m == key:
                            #print("MATCH")
                            #print("NEW LINK TO BE CREATED BETWEEN: ")
                            #print(kk)
                            #print(" AND ")
                            #print(key)
                            duplicated_link = {}
                            duplicated_link['sourcePort'] = link['sourcePort']
                            duplicated_link['targetPort'] = link['targetPort']
                            duplicated_link['source'] = kk               
                            duplicated_link['target'] = key
                            duplicated_link['key'] = count*1000
                            print(duplicated_link)
                            count = count + 1
                            duplicated_links_to_be_added.append(duplicated_link)

                            #OUTPUT LINK NEEDS TO BE ESTABLISHED AS WELL, in case not done yet
                            if key not in output_completed:
                                duplicated_link = {}
                                duplicated_link['source'] = key
                                duplicated_link['target'] = kk
                                duplicated_link['sourcePort'] = link['sourcePort']+str(count)

                                #find the correct output/result port of the node one level higher in the hierarchy
                                targetPort = find_output_port_for_node(kk, links_data)
                                #print("TARGETPORT: ")
                                #print(targetPort)
                                duplicated_link['targetPort'] = targetPort
                                print(duplicated_link)
                                count = count + 1
                                duplicated_links_to_be_added.append(duplicated_link)
                                output_completed.append(key)

    #print(duplicated_links_to_be_added)
    return(duplicated_links_to_be_added)                          

In [18]:
def find_output_port_for_node(node, links_data):
    links = links_data["links"]
    for item in links:
        if item['source'] == node:
            return item['sourcePort']

In [19]:
#MAIN
#====

In [20]:
#1. Find layer-count in the collapsible hierarchy
layer_count = find_layer_count()
print(layer_count)

3


In [21]:
#2. Find all compositions
compositions = find_compositions()
print(compositions)

[rdflib.term.URIRef('http://www.example.com#sum3Composition'), rdflib.term.URIRef('http://www.example.com#sumComposition')]


In [22]:
#3. Initialise the hierarchy between the start-nodes

node_hierarchy = {}
function_input_parameter_mapping = {}
node_function_mapping = {}

for composition in compositions:
    h = find_start_node_per_layer(composition)
    for key, value in h.items():
        # Access key and value
        node_hierarchy[key] = [value]
        next = expand_layer_with_next_nodes(value)
        while next is not None:
            node_hierarchy[key].append(next)
            next = expand_layer_with_next_nodes(next)

print(node_hierarchy)

{rdflib.term.URIRef('http://www.example.com#sum3'): [rdflib.term.URIRef('http://www.example.com#sum_2'), rdflib.term.URIRef('http://www.example.com#sum_1')], rdflib.term.URIRef('http://www.example.com#sum'): [rdflib.term.URIRef('http://www.example.com#op_add_1')]}


In [23]:
#4. Checking for additional composition decompositions (cf. triple_sum > simple_dum > op_add)
###########
#IMPORTANT#
###########

#It seems that child-nodes cannot be linked to multiple parent-nodes. Thus, rather than re-using the existing nodes, we'll have to duplicate nodes if they are re-used in multiple 'applies' relations.

temp = {}
duplicate_function_mapping = {}

#print("App\nFunction")
for key in node_hierarchy:
    for v in node_hierarchy[key]:
        #print(key)
        application,  node = find_applies_for_node(v)
        #print("\t"+application)
        #print("\t"+node)

        temp = expand_reused_composition(application, node, temp)
        node_function_mapping[application] = node

#print()
#print(temp)
#print()

count = 1;
for key, value in temp.items():
    print("\t"+key)
    newSet = set()
    for val in value:
        print("\t\t"+val)
        newSet.add(val+"_"+str(count))       

        #CREATE MAPPING BETWEEN DUPLICATED FUNCTION AND ORIGINAL FUNCTION
        #Create mapping from the original (e.g. ex:op_add_1 to a list with e.g. [ex:op_add_1_1, ex:op_add_1_2])
        duplicate_function_mapping[val+"_"+str(count)] = val

        count = count + 1
    
    node_hierarchy[key] = newSet
    
print(node_hierarchy)
print()
print(node_function_mapping)
print()
print(duplicate_function_mapping)
print()

	http://www.example.com#sum_2
		http://www.example.com#op_add_1
	http://www.example.com#sum_1
		http://www.example.com#op_add_1
{rdflib.term.URIRef('http://www.example.com#sum3'): [rdflib.term.URIRef('http://www.example.com#sum_2'), rdflib.term.URIRef('http://www.example.com#sum_1')], rdflib.term.URIRef('http://www.example.com#sum'): [rdflib.term.URIRef('http://www.example.com#op_add_1')], rdflib.term.URIRef('http://www.example.com#sum_2'): {rdflib.term.URIRef('http://www.example.com#op_add_1_1')}, rdflib.term.URIRef('http://www.example.com#sum_1'): {rdflib.term.URIRef('http://www.example.com#op_add_1_2')}}

{rdflib.term.URIRef('http://www.example.com#sum_2'): rdflib.term.URIRef('http://www.example.com#sum'), rdflib.term.URIRef('http://www.example.com#sum_1'): rdflib.term.URIRef('http://www.example.com#sum'), rdflib.term.URIRef('http://www.example.com#op_add_1'): rdflib.term.URIRef('http://www.example.com#op_add')}

{rdflib.term.URIRef('http://www.example.com#op_add_1_1'): rdflib.term.

In [24]:
#5. ELKING those nodes
nodes_data = {}
nodes_data["directed"] = True
nodes_data["graph"] = {}
nodes_data["multigraph"] = False

nodes_data = create_hier_tree(node_hierarchy, nodes_data)

print(nodes_data)

{'directed': True, 'graph': {}, 'multigraph': False, 'nodes': [{'id': rdflib.term.URIRef('http://www.example.com#sum_2')}, {'id': rdflib.term.URIRef('http://www.example.com#op_add_1_2')}, {'id': rdflib.term.URIRef('http://www.example.com#sum')}, {'id': rdflib.term.URIRef('http://www.example.com#sum_1')}, {'id': rdflib.term.URIRef('http://www.example.com#sum3')}, {'hidden': True, 'id': 'dummy'}], 'links': [{'source': rdflib.term.URIRef('http://www.example.com#sum3'), 'target': rdflib.term.URIRef('http://www.example.com#sum_2')}, {'source': rdflib.term.URIRef('http://www.example.com#sum3'), 'target': rdflib.term.URIRef('http://www.example.com#sum_1')}, {'source': rdflib.term.URIRef('http://www.example.com#sum'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1')}, {'source': rdflib.term.URIRef('http://www.example.com#sum_2'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1_1')}, {'source': rdflib.term.URIRef('http://www.example.com#sum_1'), 'target': rdflib.ter

In [25]:
#7. Creating an inventory of all ports+labels to be created

labels = create_node_ports_with_labels()
print(labels)

n77e3f47526404212b73b415687c09f6cb21
	http://www.example.com#sum3Parameter0
	a
n77e3f47526404212b73b415687c09f6cb25
	http://www.example.com#sum3Parameter1
	b
n77e3f47526404212b73b415687c09f6cb26
	http://www.example.com#sum3Parameter2
	c
n77e3f47526404212b73b415687c09f6cb38
	http://www.example.com#sumParameter1
	b
n77e3f47526404212b73b415687c09f6cb39
	http://www.example.com#sumParameter0
	a
n77e3f47526404212b73b415687c09f6cb4
	http://www.example.com#op_addOutput
n77e3f47526404212b73b415687c09f6cb27
	http://www.example.com#sum3Output
n77e3f47526404212b73b415687c09f6cb42
	http://www.example.com#sumOutput
{rdflib.term.URIRef('http://www.example.com#sum3Parameter0'): 'a', rdflib.term.URIRef('http://www.example.com#sum3Parameter1'): 'b', rdflib.term.URIRef('http://www.example.com#sum3Parameter2'): 'c', rdflib.term.URIRef('http://www.example.com#sumParameter1'): 'b', rdflib.term.URIRef('http://www.example.com#sumParameter0'): 'a', rdflib.term.URIRef('http://www.example.com#op_addOutput'): 'ou

In [26]:
#8. Establishing the dataflow links
links_data = {}
links_data["directed"] = True
links_data["graph"] = {}
links_data["multigraph"] = True
links_data["nodes"] = []
links_data["links"] = []

temp = find_mappings_and_create_links()
#print("TEMP:")
#print(temp)
links_data["links"] = temp
print(links_data)


COMPOSITION: 
http://www.example.com#sum3Composition
----------------------------------
MAPPING: n77e3f47526404212b73b415687c09f6cb8
sourcePort
	http://www.example.com#sum3Parameter1
targetPort
	http://www.example.com#sumParameter1
source
	http://www.example.com#sum3
target
	http://www.example.com#sum_2

COMPOSITION: 
http://www.example.com#sum3Composition
----------------------------------
MAPPING: n77e3f47526404212b73b415687c09f6cb11
sourcePort
	http://www.example.com#sum3Parameter2
targetPort
	http://www.example.com#sumParameter1
source
	http://www.example.com#sum3
target
	http://www.example.com#sum_1

COMPOSITION: 
http://www.example.com#sum3Composition
----------------------------------
MAPPING: n77e3f47526404212b73b415687c09f6cb14
sourcePort
	http://www.example.com#sum3Parameter0
targetPort
	http://www.example.com#sumParameter0
source
	http://www.example.com#sum3
target
	http://www.example.com#sum_2

COMPOSITION: 
http://www.example.com#sumComposition
---------------------------

In [27]:
#9. Ensure that the artifically added nodes are interlinked appropriately

new_links_to_be_added = check_and_percolate_dupplications(duplicate_function_mapping, links_data, node_hierarchy)
#print(new_links_to_be_added)
final_links_list = links_data["links"]

for item in new_links_to_be_added:
    final_links_list.append(item)

#print(links_data)

{'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter1'), 'targetPort': rdflib.term.URIRef('http://www.example.com#op_addParameter1'), 'source': rdflib.term.URIRef('http://www.example.com#sum_2'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1_1'), 'key': 1000}
{'source': rdflib.term.URIRef('http://www.example.com#op_add_1_1'), 'target': rdflib.term.URIRef('http://www.example.com#sum_2'), 'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter12'), 'targetPort': rdflib.term.URIRef('http://www.example.com#sumOutput')}
{'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter0'), 'targetPort': rdflib.term.URIRef('http://www.example.com#op_addParameter0'), 'source': rdflib.term.URIRef('http://www.example.com#sum_2'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1_1'), 'key': 3000}
{'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter1'), 'targetPort': rdflib.term.URIRef('http://www.example.com#op_ad

In [28]:
#10. Persisting the ELK JSON for the collapsible hierarchy

with open('nodes.json', 'w') as file1:
    json.dump(nodes_data, file1)

with open('links.json', 'w') as file2:
    json.dump(links_data, file2)

In [29]:
#EXAMPLE FROM THE WEB
#====================
import networkx
NX_VINFO = tuple(map(int, networkx.__version__.split(".")[:2]))
NX_EDGES = "edges" if NX_VINFO >= (3, 4) else "link"
from ipyelk.tools import ToggleCollapsedTool

def a_hierarchical_elk_example(
    tree: networkx.MultiDiGraph = None, ports: networkx.MultiDiGraph = None
):
    tree = tree or load_nx_graph("nodes.json")
    ports = ports or load_nx_graph("links.json")
    elk = ipyelk.from_nx(
        graph=ports,
        hierarchy=tree,
        layout=dict(
            min_height="200px",
            height="100%",
        ),
    )
    return elk

In [30]:
def load_nx_graph(filename: str) -> networkx.MultiDiGraph:
    return networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path(filename).resolve().read_text(encoding="utf-8")),
        **{NX_EDGES: "links"},
    )

In [31]:
def a_collapsible_elk_example(elk=None):
    elk = elk or a_hierarchical_elk_example()

    collapser = elk.get_tool(ToggleCollapsedTool)
    toggle = ipywidgets.Button(description="Toggle")
    toggle.on_click(collapser.handler)

    box = ipywidgets.VBox(
        [
            ipywidgets.HBox([
                ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"),
                toggle,
            ]),
            elk,
        ],
        layout=dict(
            min_height="500px",
            height="100%",
        ),
    )
    return box, elk

In [32]:
hier_box, hier_elk = a_collapsible_elk_example()
display(hier_box)

In [33]:
#SCRATCHPAD
#=================================================================================

In [34]:
#TEST find_layer_count
print(find_layer_count())

3


In [35]:
#TEST find_compositions
print(find_compositions())

[rdflib.term.URIRef('http://www.example.com#sum3Composition'), rdflib.term.URIRef('http://www.example.com#sumComposition')]


In [36]:
#TEST find_start_node_per_layer
print(find_start_node_per_layer(compositions[0]))

{rdflib.term.URIRef('http://www.example.com#sum3'): rdflib.term.URIRef('http://www.example.com#sum_2')}


In [37]:
def find_function_definition_for_node(node):
    query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?node fnoc:applies ?function .
        } 
        """

    for row in g.query(query, initBindings={'node': node}):
        return row.function

In [38]:
def find_input_parameters_for_function(function):
    query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?x fnoc:constituentFunction ?function .
            ?x fnoc:functionParameter ?parameter
        } 
        """

    print(function)
    function_input_parameter_mapping[function] = set()
    for row in g.query(query, initBindings={'function': function}):
        print("\t"+row.parameter)
        function_input_parameter_mapping[function].add(row.parameter)

    return function_input_parameter_mapping

In [39]:
#TEST

In [40]:
#TEST
print(create_hier_tree(node_hierarchy, {}))

{'nodes': [{'id': rdflib.term.URIRef('http://www.example.com#sum_2')}, {'id': rdflib.term.URIRef('http://www.example.com#op_add_1_2')}, {'id': rdflib.term.URIRef('http://www.example.com#sum')}, {'id': rdflib.term.URIRef('http://www.example.com#sum_1')}, {'id': rdflib.term.URIRef('http://www.example.com#sum3')}, {'hidden': True, 'id': 'dummy'}], 'links': [{'source': rdflib.term.URIRef('http://www.example.com#sum3'), 'target': rdflib.term.URIRef('http://www.example.com#sum_2')}, {'source': rdflib.term.URIRef('http://www.example.com#sum3'), 'target': rdflib.term.URIRef('http://www.example.com#sum_1')}, {'source': rdflib.term.URIRef('http://www.example.com#sum'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1')}, {'source': rdflib.term.URIRef('http://www.example.com#sum_2'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1_1')}, {'source': rdflib.term.URIRef('http://www.example.com#sum_1'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1_2')}]}


In [41]:
#TEST

In [42]:
#TEST

In [43]:
#TEST
create_node_ports_with_labels()

n77e3f47526404212b73b415687c09f6cb21
	http://www.example.com#sum3Parameter0
	a
n77e3f47526404212b73b415687c09f6cb25
	http://www.example.com#sum3Parameter1
	b
n77e3f47526404212b73b415687c09f6cb26
	http://www.example.com#sum3Parameter2
	c
n77e3f47526404212b73b415687c09f6cb38
	http://www.example.com#sumParameter1
	b
n77e3f47526404212b73b415687c09f6cb39
	http://www.example.com#sumParameter0
	a
n77e3f47526404212b73b415687c09f6cb4
	http://www.example.com#op_addOutput
n77e3f47526404212b73b415687c09f6cb27
	http://www.example.com#sum3Output
n77e3f47526404212b73b415687c09f6cb42
	http://www.example.com#sumOutput


{rdflib.term.URIRef('http://www.example.com#sum3Parameter0'): 'a',
 rdflib.term.URIRef('http://www.example.com#sum3Parameter1'): 'b',
 rdflib.term.URIRef('http://www.example.com#sum3Parameter2'): 'c',
 rdflib.term.URIRef('http://www.example.com#sumParameter1'): 'b',
 rdflib.term.URIRef('http://www.example.com#sumParameter0'): 'a',
 rdflib.term.URIRef('http://www.example.com#op_addOutput'): 'out',
 rdflib.term.URIRef('http://www.example.com#sum3Output'): 'out',
 rdflib.term.URIRef('http://www.example.com#sumOutput'): 'out'}

In [44]:
def link_parameter_flow(node_hierarchy, links_data, source, target):
    print("S")
    print(source)
    find_composition_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?composition fnoc:represents ?source
        } 
        """    

    for row in g.query(find_composition_query):
        print("C")
        print(row.composition)
        #CASE 1: DATA-FLOW between HIERARCHICAL LEVELS INPUT
        links_data = link_parameter_flow_for_source_target_in_composition(node_hierarchy, links_data, source, target, row.composition)
        #print("LINKS DATA AFTER CASE 1:")
        #print(links_data)
        #CASE 2: DATA-FLOW between HIERARCHICAL LEVELS OUTPUT
        #links_data = link_output_flow_for_soure_target_in_composition(node_hierarchy, links_data, target, source, row.composition)
        #CASE 3: DATA-FLOW on the same HIERARCHICAL LEVEL OUTPUT to INPUT (the two entities(nodes) should be checked upon in the node_hierarchy tree to check whether they're not hierarchically connected
        #links_data = link_input_output_flow_in_composition(node_hierarchy, links_data, row.composition)
    
    return links_data

In [45]:
#composition_function_mapping = {}
#function_parameter_property_mapping = {}
#function_parameter_position_mapping = {}

def link_parameter_flow_for_source_target_in_composition(node_hierarchy, links_data, source, target, composition):
    print("[link_parameter_flow_for_source_target_in_composition] S:")
    print(source)
    parameter_input_query = """
        PREFIX fno: <https://w3id.org/function/ontology#> 
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition#> 
        PREFIX ex: <http://www.example.com#>
        PREFIX prov: <http://www.w3.org/ns/prov#> 
        SELECT *
        WHERE {
            ?composition a fno:Composition .
            ?composition fnoc:composedOf ?mapping .
            ?mapping fnoc:mapFrom ?from .
            ?from fnoc:constituentFunction ?function .
            ?from fnoc:functionParameter ?parameter            
        } 
        """

    for row in g.query(parameter_input_query):
        print(row.composition)
        print("\t"+row.function)
        print("\t"+row.parameter)

        if row.composition not in composition_function_mapping.keys():
            composition_function_mapping[row.composition] = set()
        composition_function_mapping[row.composition].add(row.function)

        if row.function not in function_parameter_property_mapping.keys():
            function_parameter_property_mapping[row.function] = set()
        function_parameter_property_mapping[row.function].add(row.parameter)

    print("INFORMATION REPOSITORY")
    print(composition_function_mapping)
    print(function_parameter_property_mapping)
    print("========================")

In [46]:
#Method to initialise the port definitions on the nodes/TBD: between the nodes
#==============================================================================

def create_port_port_elk(char, input_links, links_data):
    #Create the necessary links
    print("=============================================")
    print(links_data)
    print("=============================================")
    links = links_data["links"]
    print(links)
    print("=============================================")
    index = 0;
    
    for key in input_links.keys():
        print("\nkey")
        print(key)
        child = { "sourcePort": char, "targetPort": char, "source": source, "target": target, "key": index }
        index = index +1
        links.append(child)
        ## converting char into int
        i = ord(char[0])

        ## we can add any number if we want
        ## incrementing
        i += 1

        ## casting the resultant int to char
        ## we will get 'u'
        char = chr(i)

    print(links)
    links_data["links"] = links

    return links_data

In [47]:
print(check_and_percolate_dupplications(duplicate_function_mapping, links_data, node_hierarchy))

{'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter1'), 'targetPort': rdflib.term.URIRef('http://www.example.com#op_addParameter1'), 'source': rdflib.term.URIRef('http://www.example.com#sum_2'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1_1'), 'key': 1000}
{'source': rdflib.term.URIRef('http://www.example.com#op_add_1_1'), 'target': rdflib.term.URIRef('http://www.example.com#sum_2'), 'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter12'), 'targetPort': rdflib.term.URIRef('http://www.example.com#sumOutput')}
{'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter0'), 'targetPort': rdflib.term.URIRef('http://www.example.com#op_addParameter0'), 'source': rdflib.term.URIRef('http://www.example.com#sum_2'), 'target': rdflib.term.URIRef('http://www.example.com#op_add_1_1'), 'key': 3000}
{'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter1'), 'targetPort': rdflib.term.URIRef('http://www.example.com#op_ad

In [48]:
#TEST
find_mappings_and_create_links()


COMPOSITION: 
http://www.example.com#sum3Composition
----------------------------------
MAPPING: n77e3f47526404212b73b415687c09f6cb8
sourcePort
	http://www.example.com#sum3Parameter1
targetPort
	http://www.example.com#sumParameter1
source
	http://www.example.com#sum3
target
	http://www.example.com#sum_2

COMPOSITION: 
http://www.example.com#sum3Composition
----------------------------------
MAPPING: n77e3f47526404212b73b415687c09f6cb11
sourcePort
	http://www.example.com#sum3Parameter2
targetPort
	http://www.example.com#sumParameter1
source
	http://www.example.com#sum3
target
	http://www.example.com#sum_1

COMPOSITION: 
http://www.example.com#sum3Composition
----------------------------------
MAPPING: n77e3f47526404212b73b415687c09f6cb14
sourcePort
	http://www.example.com#sum3Parameter0
targetPort
	http://www.example.com#sumParameter0
source
	http://www.example.com#sum3
target
	http://www.example.com#sum_2

COMPOSITION: 
http://www.example.com#sumComposition
---------------------------

[{'key': 0,
  'sourcePort': rdflib.term.URIRef('http://www.example.com#sum3Parameter1'),
  'targetPort': rdflib.term.URIRef('http://www.example.com#sumParameter1'),
  'source': rdflib.term.URIRef('http://www.example.com#sum3'),
  'target': rdflib.term.URIRef('http://www.example.com#sum_2')},
 {'key': 1,
  'sourcePort': rdflib.term.URIRef('http://www.example.com#sum3Parameter2'),
  'targetPort': rdflib.term.URIRef('http://www.example.com#sumParameter1'),
  'source': rdflib.term.URIRef('http://www.example.com#sum3'),
  'target': rdflib.term.URIRef('http://www.example.com#sum_1')},
 {'key': 2,
  'sourcePort': rdflib.term.URIRef('http://www.example.com#sum3Parameter0'),
  'targetPort': rdflib.term.URIRef('http://www.example.com#sumParameter0'),
  'source': rdflib.term.URIRef('http://www.example.com#sum3'),
  'target': rdflib.term.URIRef('http://www.example.com#sum_2')},
 {'key': 3,
  'sourcePort': rdflib.term.URIRef('http://www.example.com#sumParameter1'),
  'targetPort': rdflib.term.URIRe

In [49]:
#BACKUP!!

def check_and_percolate_dupplications(duplicate_function_mapping, links_data, node_hierarchy):
    #print(duplicate_function_mapping)
    #print()
    duplicated_links_to_be_added = []
    count = 1
    
    for key, value in duplicate_function_mapping.items():
        print("CONSIDERING FOLLOWING KEY/VALUE PAIR")
        print(key)
        print(value)

        for link in links_data["links"]:
            print("CHECKING LINK: ")
            print(link)
            if link['target'] == value:
                print("FOUND LINK:")
                print(link)


                ##Find the correct source reference in node_hierarchy
                for kk, vv in node_hierarchy.items():
                    print("CHECKING NODE HIERARCHY")
                    print(kk)
                    for m in vv:
                        print(m)
                        if m == key:
                            print("MATCH")
                            print("NEW LINK TO BE CREATED BETWEEN: ")
                            print(kk)
                            print(" AND ")
                            print(key)
                            duplicated_link = {}
                            duplicated_link['sourcePort'] = link['sourcePort']
                            duplicated_link['targetPort'] = link['targetPort']
                            duplicated_link['source'] = kk               
                            duplicated_link['target'] = key
                            duplicated_link['key'] = count*1000
                            print(duplicated_link)
                            count = count + 1
                            duplicated_links_to_be_added.append(duplicated_link)

    
    print(duplicated_links_to_be_added)
    return(duplicated_links_to_be_added)